# Make tables: calculate p-values and differential expressions for each cancer 

Create a dataframe with p-value results from t-tests for all proteins (trans proteomics when PTEN has cnv deletions compared to PTEN wildtype). The dataframe also includes the change in medians between deletions and wildtype. Prepare these tables for further analysis by creating csv files.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import re

import cptac
import cptac.utils as u
import plot_utils as p

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Returns a dataframe with proteomics and mutation type

# all_prot: list of trans genes

def all_prot_format_df(cancer_object, all_prot, gene_in = 'PTEN', utils = u):
    mut_type = cancer_object.get_genotype_all_vars(gene_in)
    
    if cancer_object.get_cancer_type() not in ('luad'):
        # Keep only tumor samples from proteomics
        prot_and_mutations = cancer_object.join_omics_to_mutations(
            mutations_genes = [gene_in], omics_df_name = 'proteomics', omics_genes = all_prot, tissue_type = 'tumor')
        # Reduce a multiindex 
        if isinstance(prot_and_mutations.keys(), pd.core.indexes.multi.MultiIndex):
            prot_and_mutations = utils.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)
            prot_and_mutations = rename_duplicate_cols(prot_and_mutations)
        
        # Merge Mutation column from get_genotype_all_vars (includes cnv) with proteomics
        mut_type = mut_type[['Mutation']] 
        prot_df = prot_and_mutations.iloc[:,:-4] # Keep only proteomics
        merged = prot_df.join(mut_type)
        
        # Keep only Wildtype and deletion
        compare = ['Wildtype_Tumor','Deletion']
        get = merged['Mutation'].isin(compare)
        mut_wt = merged[get]
    
    
    # Luad has no somatic mutations for PTEN which changes some things
    elif cancer_object.get_cancer_type() in ('luad'): 
        # get_genotype_all_vars add cnv data under the column PTEN
        mut_type = mut_type.drop(columns= gene_in)
        # different code because no somatic mutation data for pten (can't join to somatic mutations)
        omics = cancer_object.join_omics_to_omics(df1_name = 'CNV', df2_name='proteomics',genes1= gene_in, 
            genes2= all_prot, tissue_type = 'tumor')
        omics = utils.reduce_multiindex(omics, levels_to_drop = 1, flatten = True)
        omics = rename_duplicate_cols(omics)
        omics = omics.drop(columns='PTEN_CNV')
       
        # Merge Mutation column from get_genotype_all_vars (includes cnv) with proteomics
        merged = omics.join(mut_type) 
        # Keep only No_Mutation (wildtype) and deletion
        compare = ['No_Mutation','Deletion']
        get = merged['Mutation'].isin(compare)
        mut_wt = merged[get]
        mut_wt['Mutation'] = np.where(
            mut_wt['Mutation'] == 'No_Mutation', 'Wildtype_Tumor', 'Deletion')
    
    mut_wt = mut_wt.rename(columns={col: re.sub(r'_proteomics', '', col) for col in mut_wt.columns.tolist()})
    return mut_wt


In [4]:
'''
Params
cancer_object: Object. Variable for the loaded cancer data set.
cancer_name: Str. name to add to the created dataframe.
del_wt_df: DataFrame. Only samples with deletions and wildtype for PTEN. 
all_prot_list: List. All proteins in proteomics data frame. 

Returns a dataframe with the difference in medians between proteomics with PTEN wt and del (wt - del). 
'''

def get_change_in_medians_df(cancer_object, cancer_name, del_wt_df, all_prot_list):
    if  cancer_object.get_cancer_type() in ('endometrial'):
        d = del_wt_df[del_wt_df.Mutation == "Truncation"]
        wt = del_wt_df[del_wt_df.Mutation == "Wildtype_Tumor"]
    else:
        d = del_wt_df[del_wt_df.Mutation == "Deletion"]
        wt = del_wt_df[del_wt_df.Mutation == "Wildtype_Tumor"]
    del_med = d.median()
    wt_med = wt.median()

    med_dict = {}
    
    # Correlation: + is mutant up compared to wt, - is mutant down
    for prot in all_prot_list:
        if prot in del_med.index and prot in wt_med.index:
            dif = del_med[prot] - wt_med[prot]
            med_dict[prot] = dif
        
    df = pd.DataFrame.from_dict(med_dict, orient='index', columns=[cancer_name+'_Median'])
    df = df.reset_index().rename(columns={'index':'Proteomics'})
    
    return df
        

In [5]:
# Adds '_i' to duplicate col names where i is a number (increases with each duplicate)

def rename_duplicate_cols(df):
    cols = pd.Series(df.columns[:])

    for dup in cols[cols.duplicated()].unique(): 
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]

    # rename the columns with the cols list.
    df.columns=cols
    return df

#  Step 1: Create data frames with p-values and differential expressions

Each cancer needs a data frame containing only samples that have PTEN cnv deletions and PTEN wildtype with trans proteomics. Use wrap_ttest to run t-tests for all genes in the proteomic data frame. Use get_change_in_medians_df to create the data frame with differential expression values. Merge both data frames. 

Load in cancer data sets from cptac. 

In [6]:
#g = cptac.Gbm()
#en = cptac.Endometrial()
#h = cptac.Hnscc()
#l = cptac.Luad()
#ls = cptac.Lscc()
#o = cptac.Ovarian()
col = cptac.Colon()
#b = cptac.Brca()

In [7]:
# Get df with PTEN deletions and wt
cancer = col
gene = 'PTEN'
prot = cancer.get_proteomics()
#prot= u.reduce_multiindex(prot, levels_to_drop = 1)
prot = prot.drop(columns = 'PTEN') # cis effect
prot_list = list(prot.columns)

del_wt = all_prot_format_df(cancer, prot_list)
del_wt.head()

Name,A1BG,A1CF,A2M,AAAS,AACS,AAGAB,AAK1,AAMDC,AAMP,AAR2,...,ZNRD1,ZNRF2,ZPR1,ZRANB2,ZW10,ZWILCH,ZWINT,ZYX,ZZEF1,Mutation
Patient_ID,,,,,,,,,,,,,,,,,,,,,
01CO001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wildtype_Tumor
01CO005,-1.10,0.318,-0.487,0.0995,0.1550,0.1690,0.0653,-0.147,0.114,0.341,...,NaN,0.0384,0.0221,0.2500,0.0869,0.0331,NaN,-0.733,-0.2650,Deletion
01CO006,-1.12,-0.441,-0.347,-0.0029,0.0957,0.3960,-0.0363,-0.549,0.220,0.248,...,0.324,NaN,-0.3940,0.0846,0.0010,-0.3450,NaN,-0.658,0.0052,Wildtype_Tumor
01CO008,-1.20,0.160,-1.850,0.1190,-0.0924,0.0187,-0.2140,0.328,-0.282,-0.348,...,NaN,-0.3840,-0.1680,0.3570,-0.3250,0.3490,NaN,-0.821,0.2000,Wildtype_Tumor
01CO013,-1.89,0.112,-0.329,0.6700,0.1160,0.3130,-0.2380,-0.274,-0.554,0.270,...,NaN,NaN,-0.0656,-0.2950,0.4630,-0.4480,0.0493,-0.904,-0.1580,Wildtype_Tumor


In [8]:
# get ids of samples with PTEN del
df = del_wt.loc[del_wt['Mutation'] == 'Deletion']
del_ids = list(df.index)
print('total samples with PTEN deletion:', len(del_ids))

total samples with PTEN deletion: 25


In [9]:
# check how many samples are in stage 1 and have PTEN loss
clinical = cancer.get_clinical()
#print(clinical.columns)
print('total samples:', '\n')
print(clinical.Stage.value_counts(), '\n')

only_del = clinical[clinical.index.isin(del_ids)]
print('samples with PTEN loss:')
only_del.Stage.value_counts()

total samples: 

Stage III    48
Stage II     42
Stage I      12
Stage IV      8
Name: Stage, dtype: int64 

samples with PTEN loss:


Stage II     12
Stage III     8
Stage I       4
Stage IV      1
Name: Stage, dtype: int64

In [10]:
# get a list of all samples with initial stages
id_list = []

stage_list = ['Stage I', 'Stage II'] # colon
#stage_list = ['I', 'II', 'IA', 'IB', 'IIA', 'IIB'] # LSCC

for stage in stage_list:
    stage_df = clinical.loc[clinical['Stage'] == stage]
    s1 = list(stage_df.index)
    id_list += s1
len(id_list)

54

In [11]:
# Keep samples in the initial stages
initial = del_wt[del_wt.index.isin(id_list)]
initial.Mutation.value_counts()

Wildtype_Tumor    36
Deletion          16
Name: Mutation, dtype: int64

In [12]:
prot_cols = list(initial.columns[:-1])

# Get all pvals
pval = u.wrap_ttest(initial, 'Mutation', prot_cols, return_all = True, correction_method = 'fdr_bh', mincount = 5)
pval

,Comparison,P_Value
0,NAXE,0.090697
1,FAM45A,0.166665
2,FHL2,0.321489
3,NDUFAF3,0.321489
4,LTBP2,0.362640
...,...,...
6784,FAU,0.999204
6785,TRIM22,0.999204
6786,RRM2,0.999204
6787,PGLYRP2,0.999378


In [13]:
initial[['NAXE', 'Mutation']]

Name,NAXE,Mutation
Patient_ID,,
01CO005,0.1470,Deletion
01CO008,-0.2520,Wildtype_Tumor
01CO013,0.0459,Wildtype_Tumor
01CO015,0.0692,Deletion
01CO022,-0.4700,Wildtype_Tumor
05CO003,-0.1480,Wildtype_Tumor
05CO005,NaN,Wildtype_Tumor
05CO011,0.4020,Deletion
05CO015,-0.3390,Wildtype_Tumor
